In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
with open('player_salary.xml', 'r') as f:
    player_salary = f.read()

In [3]:
# parsing the data
df_salary = pd.read_xml(player_salary).drop(columns=["registerValue", "salary", "fullTeamName"])

In [4]:
len(df_salary)

11837

In [5]:
df_salary.head()

,id,name,startYear,endYear,team
0,player_salary_id_1,A.C. Green,1990,1991,LAL
1,player_salary_id_2,A.C. Green,1991,1992,LAL
2,player_salary_id_3,A.C. Green,1992,1993,LAL
3,player_salary_id_4,A.C. Green,1993,1994,PHO
4,player_salary_id_5,A.C. Green,1994,1995,PHO


In [6]:
with open('player_stat.xml', 'r') as f:
    player_stat = f.read()

In [7]:
# parsing the data
df_stat = pd.read_xml(player_stat).drop(columns=["position", "position", "height", "weight", "birth_date", "college"])

In [8]:
len(df_stat)

4550

In [9]:
df_stat.head()

,id,name,year_start,year_end
0,player_stat_id_1,Alaa Abdelnaby,1991,1995
1,player_stat_id_2,Zaid Abdul-Aziz,1969,1978
2,player_stat_id_3,Kareem Abdul-Jabbar,1970,1989
3,player_stat_id_4,Mahmoud Abdul-Rauf,1991,2001
4,player_stat_id_5,Tariq Abdul-Wahad,1998,2003


In [10]:
# use a subset of the data or the size of the cartesian product will be too large
df_salary_sample = df_salary.sample(n=2000, random_state=1)
df_stat_sample = df_stat.sample(n=2000, random_state=1)

In [11]:
# cartesian product of datasets
df = pd.merge(df_salary_sample.assign(key=1), df_stat_sample.assign(key=1), on='key').drop('key', axis=1)

In [12]:
len(df)

4000000

In [13]:
df.head()

,id_x,name_x,startYear,endYear,team,id_y,name_y,year_start,year_end
0,player_salary_id_11246,Troy Murphy,2008,2009,IND,player_stat_id_2721,Ken Menke,1950,1950
1,player_salary_id_11246,Troy Murphy,2008,2009,IND,player_stat_id_788,Joe Cooke,1971,1971
2,player_salary_id_11246,Troy Murphy,2008,2009,IND,player_stat_id_3133,Charlie Paulk,1969,1972
3,player_salary_id_11246,Troy Murphy,2008,2009,IND,player_stat_id_1724,Mark Hendrickson,1997,2000
4,player_salary_id_11246,Troy Murphy,2008,2009,IND,player_stat_id_730,Bob Cluggish,1947,1947


In [14]:
def sim(str1, str2):   
    # calculate the minimum edit distance
    len1 = len(str1) + 1
    len2 = len(str2) + 1
    d = np.ndarray(shape=(len1, len2), dtype=int, buffer=np.zeros((len1, len2)))
    for i in range(len1):
        d[i, 0] = i
    for i in range(len2):
        d[0, i] = i
    for i in range(1, len1):
        for j in range(1, len2):
            if str1[i - 1] == str2[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                d[i][j] = min(d[i - 1][j] + 1, d[i][j - 1] + 1, d[i - 1][j - 1] + 1)
    
    # similarity
    return 1 - d[len1 - 1][len2 - 1] / max(len1, len2)

In [15]:
%%time

df["sim"] = df.apply(lambda x: sim(x['name_x'].lower(), x['name_y'].lower()), axis=1)

CPU times: user 18min 26s, sys: 15.7 s, total: 18min 42s
Wall time: 18min 36s


In [16]:
df["sim_group"] = df["sim"].round(1)

In [17]:
df["sim_group"].value_counts()

0.2    1999848
0.3     942957
0.1     696315
0.4     294901
0.5      49497
0.6      10821
0.7       3890
1.0        913
0.8        772
0.9         79
0.0          7
Name: sim_group, dtype: int64

In [18]:
# matching record pairs and some corner case matches
df1 = df[df["sim_group"] == 1.0].sample(n=50, random_state=1)
df1["result"] = True
df1["is_corner_case"] = None

In [19]:
# non matching record pairs
df2 = df[df["sim_group"] <= 0.2].sample(n=100, random_state=1)
df2["result"] = False
df2["is_corner_case"] = False

In [20]:
# corner case matches and non matches (mostly non matches)
df3 = df[(df["sim_group"] >= 0.7) & (df["sim_group"] <= 0.8)].sample(n=25, random_state=1)
df3["result"] = None
df3["is_corner_case"] = True

In [21]:
# corner case matches and non matches (mostly matches)
df4 = df[df["sim_group"] == 0.9].sample(n=25, random_state=1)
df4["result"] = None
df4["is_corner_case"] = True

In [22]:
df_gold_standard = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [23]:
df_gold_standard.head()

,id_x,name_x,startYear,endYear,team,id_y,name_y,year_start,year_end,sim,sim_group,result,is_corner_case
0,player_salary_id_11675,William Cunningham,1997,1998,PHO,player_stat_id_858,William Cunningham,1998,1999,1.0,1.0,True,None
1,player_salary_id_11411,Victor Oladipo,2016,2017,OKC,player_stat_id_3036,Victor Oladipo,2014,2018,1.0,1.0,True,None
2,player_salary_id_2935,Dell Demps,1995,1996,SAS,player_stat_id_974,Dell Demps,1994,1997,1.0,1.0,True,None
3,player_salary_id_4061,Frank Kornet,1990,1991,MIL,player_stat_id_2254,Frank Kornet,1990,1991,1.0,1.0,True,None
4,player_salary_id_8786,Paul Davis,2009,2010,WAS,player_stat_id_932,Paul Davis,2007,2010,1.0,1.0,True,None


In [24]:
# rename the column names
df_gold_standard.rename(columns={
    "id_x": "player_salary_id", "name_x": "player_salary_name",
    "startYear": "player_salary_start_year", "endYear": "player_salary_end_year",
    "team": "player_salary_team", 
    "id_y": "player_stat_id", "name_y": "player_stat_name",
    "year_start": "player_stat_start_year", "year_end": "player_stat_end_year"}, inplace=True)

In [25]:
# reorder the columns
order = [
    "player_stat_id", "player_salary_id", "player_stat_name", "player_salary_name",
    "player_stat_start_year", "player_salary_start_year", "player_stat_end_year", "player_salary_end_year",
    "player_salary_team",
    "sim", "sim_group", "result", "is_corner_case"
]
    
df_gold_standard = df_gold_standard[order]

In [26]:
df_gold_standard.head()

,player_stat_id,player_salary_id,player_stat_name,player_salary_name,player_stat_start_year,player_salary_start_year,player_stat_end_year,player_salary_end_year,player_salary_team,sim,sim_group,result,is_corner_case
0,player_stat_id_858,player_salary_id_11675,William Cunningham,William Cunningham,1998,1997,1999,1998,PHO,1.0,1.0,True,None
1,player_stat_id_3036,player_salary_id_11411,Victor Oladipo,Victor Oladipo,2014,2016,2018,2017,OKC,1.0,1.0,True,None
2,player_stat_id_974,player_salary_id_2935,Dell Demps,Dell Demps,1994,1995,1997,1996,SAS,1.0,1.0,True,None
3,player_stat_id_2254,player_salary_id_4061,Frank Kornet,Frank Kornet,1990,1990,1991,1991,MIL,1.0,1.0,True,None
4,player_stat_id_932,player_salary_id_8786,Paul Davis,Paul Davis,2007,2009,2010,2010,WAS,1.0,1.0,True,None


In [27]:
df_gold_standard.to_csv("gs_stat_2_salary.csv", index=False)